In [1]:
from libraries import *

In [12]:
df = pd.read_csv("1-25Data-cleaned.csv")
df

,Section,Url,Content,Metadata
0,10. -Front Matter,https://uscode.house.gov/view.xhtml?req=granul...,18 USC Ch. 10: Front Matter Result ...,"Amendments2004-Pub. L. 108Ã¢ÂÂ458, title VI,..."
1,175. Prohibitions with respect to biological w...,https://uscode.house.gov/view.xhtml?req=granul...,"(a) In General.-Whoever knowingly develops, pr...",Amendments2002-Subsec. (c). Pub. L. 107Ã¢ÂÂ1...
2,175. a. Requests for military assistance to en...,https://uscode.house.gov/view.xhtml?req=granul...,18 USC 175a: Requests for military assistance ...,"References in TextSection 382 of title 10, ref..."
3,175. b. Possession by restricted persons,https://uscode.house.gov/view.xhtml?req=granul...,18 USC 175b: Possession by restricted persons ...,References in TextSection 351A of the Public H...
4,175. c. Variola virus,https://uscode.house.gov/view.xhtml?req=granul...,18 USC 175c: Variola virus Result 1...,NaN
...,...,...,...,...
482,511. Altering or removing motor vehicle identi...,https://uscode.house.gov/view.xhtml?req=granul...,"(a) A person who-(1) knowingly removes, oblite...",References in TextThe Motor Vehicle Theft Prev...
483,"511A. Repealed. Pub. L. 116-260, div. O, title...",https://uscode.house.gov/view.xhtml?req=granul...,"18 USC 511A: Repealed. Pub. L. 116-260, div. O...",NaN
484,512. Forfeiture of certain motor vehicles and ...,https://uscode.house.gov/view.xhtml?req=granul...,(a) If an identification number for a motor ve...,Amendments1994-Subsec. (a)(2)(A). Pub. L. 103â...
485,513. Securities of the States and private enti...,https://uscode.house.gov/view.xhtml?req=granul...,"(a) Whoever makes, utters or possesses a count...",References in TextSection 916 of the Electroni...


In [3]:
df['Content'] = df['Content'].fillna("").astype(str)

In [6]:
max = 0
for i in range(len(df['Content'])):
    try:
        if len(df['Content'][i]) > max:
            max = len(df['Content'][i])
    except:
        print(i)        
max

96371

In [7]:
print(len(df))

487


In [8]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased") # nlpaueb/legal-bert-base-uncased   |  
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 3: Function to get embeddings
def get_embedding(text, tokenizer, model):
    # Tokenize and encode the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    
    # Compute embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the mean of the token embeddings (for sentence embedding)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# Step 4: Encode the entire 'Content' column
embeddings = np.array([get_embedding(text, tokenizer, model).cpu().numpy() for text in df['Content']]).astype('float32')

# Step 5: Initialize a FAISS index with L2 (Euclidean) distance
dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings
index = faiss.IndexFlatL2(dimension)  # IndexFlatL2 is a flat index (no compression)

# Step 6: Add the vectors to the index
index.add(embeddings)

# Optional: Verify the number of vectors in the index
print(f"Number of vectors in the index: {index.ntotal}")


Number of vectors in the index: 487


In [9]:
np.savetxt("embeddings-complete-legal-bert-base-uncased.csv", embeddings, delimiter=',')

In [12]:
# test = ''' 
#  for which the maximum fine is no greater than the amount set forth
# for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or
# (7) in the case of an organization.
# '''

# # Step 8: Encode the test query
# test_embedding = get_embedding(test, tokenizer, model).reshape(1, -1).cpu().numpy().astype('float32')

# # Step 9: Example: Search for the nearest neighbors
# D, I = index.search(test_embedding, k=2)  # Search for 2 nearest neighbors

# # Step 10: Print the distances and indices of the nearest neighbors
# print("Distances:", D)
# print("Indices:", I)

# # Optionally print the neighbor texts
# for idx in I[0]:
#     print("Neighbor text:", df['Content'].iloc[idx])

: 

Cosine Similarity for FIASS

In [ ]:
# import faiss
# import numpy as np

# # Assuming 'embeddings' contains the embeddings loaded from your CSV file
# # Step 1: Normalize the embeddings for cosine similarity
# def normalize(embeddings):
#     norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
#     return embeddings / norms

# # Step 2: Normalize the existing embeddings
# normalized_embeddings = normalize(embeddings)

# # Step 3: Initialize a FAISS index with Inner Product (for cosine similarity)
# dimension = normalized_embeddings.shape[1]
# index = faiss.IndexFlatIP(dimension)  # IndexFlatIP is for inner product (dot product)

# # Step 4: Add the normalized vectors to the index
# index.add(normalized_embeddings)

# # Optional: Verify the number of vectors in the index
# print(f"Number of vectors in the index: {index.ntotal}")

# # Step 5: Normalize the test query embedding
# test = ''' 
# for which the maximum fine is no greater than the amount set forth
# for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or
# (7) in the case of an organization.
# '''
# # Get the test embedding using the existing model and tokenizer
# test_embedding = get_embedding(test, tokenizer, model).cpu().numpy().astype('float32')
# test_embedding = normalize(test_embedding.reshape(1, -1))

# # Step 6: Search for the nearest neighbors
# D, I = index.search(test_embedding, k=2)  # Search for 2 nearest neighbors

# # Step 7: Print the distances and indices of the nearest neighbors
# print("Distances:", D)
# print("Indices:", I)

# # Optionally print the neighbor texts
# for idx in I[0]:
#     print("Neighbor text:", df['Content'].iloc[idx])
